# CLASE 05 - AFTER CLASS

## Actividad 1: Eliminación de Duplicados y Tratamiento de Datos Faltantes

**Objetivos principales:**
1. Cargar el conjunto de datos y familiarizarnos con su estructura.
2. Identificar y eliminar las filas duplicadas.
3. Manejar los datos faltantes en la columna `edad`, evaluando dos estrategias:
   - Eliminar las filas sin edad.
   - Completar el dato con la media de la columna.
4. Evaluar el impacto de cada decisión sobre el análisis de clientes.

---

### Paso 1: Cargar el conjunto de datos
En este primer paso vamos a:
- Importar las librerías necesarias (`pandas` y `numpy`).
- Cargar el archivo `data_clientes.xlsx` en un **DataFrame** de Pandas.
- Visualizar las primeras filas para familiarizarnos con la estructura del dataset.


In [21]:
# =========================================================
# Paso 1: Cargar el conjunto de datos
# =========================================================

# Importamos las librerías necesarias
import pandas as pd
import numpy as np

# Cargamos el archivo Excel en un DataFrame
df = pd.read_excel("data_clientes.xlsx")

# Mostramos las primeras filas para revisar la estructura
print("=== Primeras filas del dataset ===")
display(df.head())

# Mostramos información general sobre las columnas y tipos de datos
print("\n=== Información del DataFrame ===")
print(df.info())


=== Primeras filas del dataset ===


,Nombre,Edad,Email
0,Elonore Over,40.0,eoverh6@ocn.ne.jp
1,Jacquette Gillbe,49.0,jgillbeb8@diigo.com
2,Sheelah Olechnowicz,61.0,solechnowiczch@mlb.com
3,Morris Bilovus,32.0,mbilovus3q@artisteer.com
4,Erda Geipel,60.0,egeipelj5@jalbum.net



=== Información del DataFrame ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Nombre  1032 non-null   object 
 1   Edad    973 non-null    float64
 2   Email   1032 non-null   object 
dtypes: float64(1), object(2)
memory usage: 24.3+ KB
None


## Análisis de la salida del código anterior:

Al cargar los datos, observamos lo siguiente:

- El dataset contiene **1032 registros** y **3 columnas**:  
  - `Nombre` tipo *object* (texto).  
  - `Edad` tipo *float64* (números decimales).  
  - `Email` tipo *object* (texto).

- En la columna `Edad` se detectan **valores faltantes**:  
  - Solo 973 registros tienen edad informada.  
  - Esto significa que hay **59 valores nulos** aproximadamente.

- Las columnas `Nombre` y `Email` están completas (1032 registros no nulos).

- La primera inspección de los datos muestra que la estructura es coherente:
  - Cada fila corresponde a un cliente.
  - El dataset incluye información básica de contacto y una variable numérica (`Edad`) que será fundamental para el análisis.

---

### Conclusión inicial
El dataset es relativamente pequeño (1032 registros) y fácil de manipular en Pandas.  
Ya sabemos que tendremos que trabajar con **duplicados** y **valores faltantes en la columna `Edad`**, tal como podíamos suponer a partir de la consigna.


### Paso 2: Detección y reporte de duplicados

Antes de eliminar, **vamos a medir y documentar**. Siempre debemos estar atentos que existen dos tipos de duplicados:

1) **Duplicados exactos (fila completa):**  
   Dos filas son idénticas en **todas** las columnas (`Nombre`, `Email`, `Edad`).  
   - Detección: `df.duplicated()`  
   - Vista de los casos: `df[df.duplicated(keep=False)]`

2) **Duplicados por "clave de negocio"”:**  
   Mismo **Email** (puede variar `Edad` o haber nulos). En las bases de clientes, el email suele ser un identificador bastante bueno.  
   - Detección: `df.duplicated(subset=["Email"])`  
   - Vista de los casos: `df[df.duplicated(subset=["Email"], keep=False)]`

> Nota: También podemos chequear la dupla `["Nombre","Email"]` (o cualquier otra combinación) si lo necesitamos, pero el **Email** por sí solo suele alcanzar: si va a ser usado para comunicar novedades, la gente suele dar un correo válido.

-----

#### La consigna pide:
- **1 -** Contar y mostrar **duplicados exactos**.  
- **2 -** Contar y mostrar **duplicados** (buscaremos por Email).  
- **3 -** Preparar un **reporte**: Mostramos cuántos duplicados hay en cada caso.  
- **4 -** Eliminar duplicados:
  - `drop_duplicates()` para duplicados exactos.
  - `drop_duplicates(subset=["Email"], keep="first")` para consolidar usando la columna  `Email`.
  - Registrar lo realizado: "Quitamos X duplicados exactos y Y por Email".

Este abordaje (recomendado) nos permite **mantener la trazabilidad** (qué se detectó y qué se decidió) antes de modificar el dataset.


In [22]:
# =========================================================
# Paso 1: Identificar duplicados exactos (toda la fila igual)
# =========================================================

# Cantidad de filas duplicadas (exactas)
total_duplicados_exactos = df.duplicated().sum()

print(f"Cantidad de duplicados exactos en el dataset: {total_duplicados_exactos}")

# Mostrar ejemplos de filas duplicadas
if total_duplicados_exactos > 0:
    print("\n=== Ejemplos de duplicados exactos ===")
    display(df[df.duplicated(keep=False)].head(10))
else:
    print("No se encontraron duplicados exactos.")


Cantidad de duplicados exactos en el dataset: 32

=== Ejemplos de duplicados exactos ===


,Nombre,Edad,Email
8,Federica Shambrook,33.0,fshambrook96@cdbaby.com
18,Sindee Godbert,40.0,sgodbert8v@amazon.co.uk
31,Donella Bandy,44.0,dbandy99@freewebs.com
61,Neysa Kelland,63.0,nkelland8x@answers.com
67,Rutherford Fetterplace,65.0,rfetterplace8h@globo.com
76,Veda Sherman,33.0,vsherman8y@mac.com
86,Nevins Barford,54.0,nbarford90@e-recht24.de
94,Dewain Lindmark,66.0,dlindmark8k@chronoengine.com
122,Libbey O'Hare,53.0,lohare93@privacy.gov.au
127,Daniella Vasey,59.0,dvasey95@nydailynews.com


In [23]:
# =========================================================
# Paso 2: Identificar duplicados por Email
# =========================================================

# Cantidad de duplicados considerando solo el campo 'Email'
total_duplicados_email = df.duplicated(subset=["Email"]).sum()

print(f"Cantidad de duplicados por Email en el dataset: {total_duplicados_email}")

# Mostrar ejemplos de registros con el mismo Email
if total_duplicados_email > 0:
    print("\n=== Ejemplos de duplicados por Email ===")
    display(df[df.duplicated(subset=["Email"], keep=False)].head(10))
else:
    print("No se encontraron duplicados por Email.")


Cantidad de duplicados por Email en el dataset: 32

=== Ejemplos de duplicados por Email ===


,Nombre,Edad,Email
8,Federica Shambrook,33.0,fshambrook96@cdbaby.com
18,Sindee Godbert,40.0,sgodbert8v@amazon.co.uk
31,Donella Bandy,44.0,dbandy99@freewebs.com
61,Neysa Kelland,63.0,nkelland8x@answers.com
67,Rutherford Fetterplace,65.0,rfetterplace8h@globo.com
76,Veda Sherman,33.0,vsherman8y@mac.com
86,Nevins Barford,54.0,nbarford90@e-recht24.de
94,Dewain Lindmark,66.0,dlindmark8k@chronoengine.com
122,Libbey O'Hare,53.0,lohare93@privacy.gov.au
127,Daniella Vasey,59.0,dvasey95@nydailynews.com


In [24]:
# =========================================================
# Paso 3: Mini-reporte de duplicados
# =========================================================

# Recuento de duplicados exactos
total_duplicados_exactos = df.duplicated().sum()

# Recuento de duplicados por Email
total_duplicados_email = df.duplicated(subset=["Email"]).sum()

print("=== Reporte de duplicados ===\n")
print(f"- Duplicados exactos: {total_duplicados_exactos}")
print(f"- Duplicados por Email: {total_duplicados_email}")

# Muestra ejemplos si existen
if total_duplicados_exactos > 0:
    print("\nEjemplos de duplicados exactos:")
    display(df[df.duplicated(keep=False)].head(5))

if total_duplicados_email > 0:
    print("\nEjemplos de duplicados por Email:")
    display(df[df.duplicated(subset=['Email'], keep=False)].head(5))


=== Reporte de duplicados ===

- Duplicados exactos: 32
- Duplicados por Email: 32

Ejemplos de duplicados exactos:


,Nombre,Edad,Email
8,Federica Shambrook,33.0,fshambrook96@cdbaby.com
18,Sindee Godbert,40.0,sgodbert8v@amazon.co.uk
31,Donella Bandy,44.0,dbandy99@freewebs.com
61,Neysa Kelland,63.0,nkelland8x@answers.com
67,Rutherford Fetterplace,65.0,rfetterplace8h@globo.com



Ejemplos de duplicados por Email:


,Nombre,Edad,Email
8,Federica Shambrook,33.0,fshambrook96@cdbaby.com
18,Sindee Godbert,40.0,sgodbert8v@amazon.co.uk
31,Donella Bandy,44.0,dbandy99@freewebs.com
61,Neysa Kelland,63.0,nkelland8x@answers.com
67,Rutherford Fetterplace,65.0,rfetterplace8h@globo.com


In [25]:
# =========================================================
# Paso 4: Eliminación de duplicados
# =========================================================

# 1. Eliminamos duplicados exactos (toda la fila idéntica)
df_sin_exactos = df.drop_duplicates()
print(f"Dataset sin duplicados exactos: {len(df_sin_exactos)} filas (antes {len(df)})")

# 2. Eliminamos duplicados por Email
#    Conservamos solo la primera aparición de cada Email
df_sin_duplicados = df_sin_exactos.drop_duplicates(subset=["Email"], keep="first")
print(f"Dataset sin duplicados por Email: {len(df_sin_duplicados)} filas (antes {len(df_sin_exactos)})")

# 3. Reporte final
duplicados_eliminados = len(df) - len(df_sin_duplicados)
print(f"\nResumen:")
print(f"- Filas iniciales: {len(df)}")
print(f"- Filas finales sin duplicados: {len(df_sin_duplicados)}")
print(f"- Total de duplicados eliminados: {duplicados_eliminados}")

# 4. Mostramos las primeras filas del dataset limpio
display(df_sin_duplicados.head())


Dataset sin duplicados exactos: 1000 filas (antes 1032)
Dataset sin duplicados por Email: 1000 filas (antes 1000)

Resumen:
- Filas iniciales: 1032
- Filas finales sin duplicados: 1000
- Total de duplicados eliminados: 32


,Nombre,Edad,Email
0,Elonore Over,40.0,eoverh6@ocn.ne.jp
1,Jacquette Gillbe,49.0,jgillbeb8@diigo.com
2,Sheelah Olechnowicz,61.0,solechnowiczch@mlb.com
3,Morris Bilovus,32.0,mbilovus3q@artisteer.com
4,Erda Geipel,60.0,egeipelj5@jalbum.net


### Paso 3: Manejo de valores faltantes en la columna `Edad`

Una vez eliminados los duplicados, seguimos con el otro gran problema del dataset: **los valores nulos en `Edad`**.

Acá tenemos que determinar una estrategia, que en el "mundo real" depende en gran medida del estado inicial de los datos, y de lo que querramos analizar en ellos. En este caso:

- La edad es fundamental para segmentar a los clientes en grupos etarios.  
- Y si dejamos los nulos, el análisis quedará incompleto o sesgado.  

#### Estrategias posibles:

1. **Eliminar filas incompletas**  
   - Usando `dropna(subset=["Edad"])`.  
   - Esto nos conviene cuando el número de nulos es bajo y no compromete el tamaño de la muestra.  

2. **Imputar valores faltantes**  
   - Para variables numéricas como `Edad`, lo más común es reemplazar con la **media** o la **mediana** de la columna.  
   - Usamos `fillna(df["Edad"].mean())` o `fillna(df["Edad"].median())`.  
   - Esto permite conservar todos los registros, aunque introduce un valor "estimado".   

-----

#### Hagamos lo siguiente
- **1 -** Contar cuántos nulos hay en `Edad` y calcular su porcentaje.  
- **2 -** Crear un dataset donde eliminemos filas con `Edad` nula.  
- **3 -** Crear otro dataset donde imputemos `Edad` con la media.  
- **4 -** Comparamos ambas estrategias y reflexionamos (acá se puede poner picante la discusión) sobre el impacto en el análisis.

In [26]:
# =========================================================
# Paso 1: Conteo de valores nulos en la columna 'Edad'
# =========================================================

# Total de filas en el dataset limpio (sin duplicados)
total_filas = len(df_sin_duplicados)

# Conteo de nulos en 'Edad'
nulos_edad = df_sin_duplicados["Edad"].isna().sum()

# Porcentaje de nulos
porcentaje_nulos = (nulos_edad / total_filas) * 100

print("=== Análisis de valores nulos en 'Edad' ===")
print(f"- Total de filas: {total_filas}")
print(f"- Filas con 'Edad' nula: {nulos_edad}")
print(f"- Porcentaje de nulos: {porcentaje_nulos:.2f}%")

# Mostrar ejemplos de registros con 'Edad' nula
if nulos_edad > 0:
    print("\nEjemplos de registros con 'Edad' nula:")
    display(df_sin_duplicados[df_sin_duplicados["Edad"].isna()].head(10))
else:
    print("\nNo se encontraron nulos en la columna 'Edad'.")


=== Análisis de valores nulos en 'Edad' ===
- Total de filas: 1000
- Filas con 'Edad' nula: 57
- Porcentaje de nulos: 5.70%

Ejemplos de registros con 'Edad' nula:


,Nombre,Edad,Email
23,Timotheus Meggison,NaN,tmeggisong4@digg.com
32,Dominga Dewdney,NaN,ddewdneyrn@naver.com
44,Peggy Songer,NaN,psongeru@photobucket.com
52,Dewey McCulley,NaN,dmcculleyko@tinypic.com
53,Phil Triggol,NaN,ptriggol2l@redcross.org
74,Lilyan Dulton,NaN,ldulton7e@devhub.com
78,Janina Florence,NaN,jflorencekl@cdc.gov
88,Del Lente,NaN,dlente7m@nature.com
110,Wilone Sked,NaN,wskedg7@intel.com
151,Ronnica Mabey,NaN,rmabey8w@topsy.com


In [27]:
# =========================================================
# Paso 2: Eliminar filas con 'Edad' nula
# =========================================================

# Creamos una copia del dataset sin registros con 'Edad' nula
df_sin_nulos = df_sin_duplicados.dropna(subset=["Edad"])

print("=== Eliminación de filas con 'Edad' nula ===")
print(f"- Filas antes: {len(df_sin_duplicados)}")
print(f"- Filas después: {len(df_sin_nulos)}")
print(f"- Filas eliminadas: {len(df_sin_duplicados) - len(df_sin_nulos)}")

# Mostramos las primeras filas del nuevo dataset
display(df_sin_nulos.head())


=== Eliminación de filas con 'Edad' nula ===
- Filas antes: 1000
- Filas después: 943
- Filas eliminadas: 57


,Nombre,Edad,Email
0,Elonore Over,40.0,eoverh6@ocn.ne.jp
1,Jacquette Gillbe,49.0,jgillbeb8@diigo.com
2,Sheelah Olechnowicz,61.0,solechnowiczch@mlb.com
3,Morris Bilovus,32.0,mbilovus3q@artisteer.com
4,Erda Geipel,60.0,egeipelj5@jalbum.net


In [28]:
# =========================================================
# Paso 3: Imputar valores faltantes en 'Edad' con la media
# =========================================================

# Calculamos la media de la columna 'Edad' (ignorando nulos)
media_edad = df_sin_duplicados["Edad"].mean()

print(f"Media de 'Edad' calculada: {media_edad:.2f}")

# Creamos una copia del dataset imputando valores faltantes
df_imputado_media = df_sin_duplicados.copy()
df_imputado_media["Edad"] = df_imputado_media["Edad"].fillna(media_edad)

print("\n=== Imputación completada ===")
print(f"- Filas antes: {len(df_sin_duplicados)}")
print(f"- Filas después: {len(df_imputado_media)} (ninguna fila eliminada)")

# Mostramos ejemplos de registros que tenían 'Edad' nula (ahora imputada)
print("\nEjemplos de registros imputados:")
display(df_imputado_media.loc[df_sin_duplicados["Edad"].isna()].head())


Media de 'Edad' calculada: 46.47

=== Imputación completada ===
- Filas antes: 1000
- Filas después: 1000 (ninguna fila eliminada)

Ejemplos de registros imputados:


,Nombre,Edad,Email
23,Timotheus Meggison,46.466596,tmeggisong4@digg.com
32,Dominga Dewdney,46.466596,ddewdneyrn@naver.com
44,Peggy Songer,46.466596,psongeru@photobucket.com
52,Dewey McCulley,46.466596,dmcculleyko@tinypic.com
53,Phil Triggol,46.466596,ptriggol2l@redcross.org


In [29]:
# =========================================================
# Paso 4: Comparar estrategias de manejo de nulos en 'Edad'
# =========================================================

print("=== Comparación de Estrategias ===\n")

# 1. Eliminación de filas
filas_eliminadas = len(df_sin_duplicados) - len(df_sin_nulos)
print(f"1) Eliminación de filas con 'Edad' nula:")
print(f"- Filas iniciales: {len(df_sin_duplicados)}")
print(f"- Filas después: {len(df_sin_nulos)}")
print(f"- Filas eliminadas: {filas_eliminadas}")
print("Ventaja: dataset más 'real' (solo datos válidos).")
print("Desventaja: se pierde información de clientes.\n")

# 2. Imputación con la media
nulos_restantes = df_imputado_media["Edad"].isna().sum()
print(f"2) Imputación con la media de 'Edad':")
print(f"- Filas iniciales: {len(df_sin_duplicados)}")
print(f"- Filas después: {len(df_imputado_media)} (ninguna fila eliminada)")
print(f"- Nulos restantes en 'Edad': {nulos_restantes}")
print("Ventaja: se conserva la cantidad total de clientes.")
print("Desventaja: se introduce un valor estimado que puede distorsionar la distribución de edades.\n")

# 3. Estadísticas descriptivas para comparar
print("=== Estadísticas descriptivas de 'Edad' ===")
print("\nDataset sin nulos:")
print(df_sin_nulos["Edad"].describe())

print("\nDataset imputado con media:")
print(df_imputado_media["Edad"].describe())


=== Comparación de Estrategias ===

1) Eliminación de filas con 'Edad' nula:
- Filas iniciales: 1000
- Filas después: 943
- Filas eliminadas: 57
Ventaja: dataset más 'real' (solo datos válidos).
Desventaja: se pierde información de clientes.

2) Imputación con la media de 'Edad':
- Filas iniciales: 1000
- Filas después: 1000 (ninguna fila eliminada)
- Nulos restantes en 'Edad': 0
Ventaja: se conserva la cantidad total de clientes.
Desventaja: se introduce un valor estimado que puede distorsionar la distribución de edades.

=== Estadísticas descriptivas de 'Edad' ===

Dataset sin nulos:
count    943.000000
mean      46.466596
std       12.311711
min       25.000000
25%       36.000000
50%       47.000000
75%       57.000000
max       67.000000
Name: Edad, dtype: float64

Dataset imputado con media:
count    1000.000000
mean       46.466596
std        11.955318
min        25.000000
25%        37.000000
50%        46.466596
75%        57.000000
max        67.000000
Name: Edad, dtype: floa

# Cierre

Trabajamos en la **limpieza de datos** sobre el dataset de clientes.  
El proceso incluyó dos tareas principales: **eliminación de duplicados** y **manejo de valores faltantes en `Edad`**.

---

## Eliminación de duplicados
- Se detectaron duplicados **exactos** y **por Email**.
- Se aplicó `drop_duplicates()`:
  - Primero para eliminar duplicados exactos.
  - Luego sobre `Email`, conservando la primera aparición de cada cliente.
- Resultado: se redujo el dataset a una versión **sin registros repetidos**, garantizando consistencia.

---

## Manejo de valores faltantes en `Edad`
Probamos dos estrategias:

1. **Eliminar filas con `Edad` nula**  
   - Se usó `dropna(subset=["Edad"])`.  
   - Ventaja: dataset más confiable (solo valores reales).  
   - Desventaja: pérdida de información (clientes descartados).

2. **Imputar valores nulos con la media**  
   - Se usó `fillna(df["Edad"].mean())`.  
   - Ventaja: se conserva el 100% de los registros.  
   - Desventaja: los valores imputados no representan clientes reales y pueden distorsionar la distribución.

---

## Conclusiones
- La **estrategia óptima** depende del contexto y del objetivo del análisis:
  - Si necesitamos máxima precisión en segmentación etaria => conviene **eliminar** filas incompletas.  
  - Si es más importante conservar la base completa de clientes para otras variables => conviene **imputar**.  
- En ambos casos, **dejamos documentado** cuántos registros se modificaron/eliminaron, lo que garantiza trazabilidad.



# Actividad 2: Corrección de Tipos de Datos y Normalización

**Objetivos:**
1. Cargar el conjunto de datos de productos para familiarizarnos con su estructura.
2. Detectar problemas de **tipos de datos** (por ejemplo, precios almacenados como texto).
3. Corregir los tipos de datos usando Pandas (`astype`, `to_numeric`, `to_datetime`).
4. Normalizar campos de texto para asegurar consistencia (pasar a minúsculas, eliminar caracteres especiales, unificar categorías).
5. Documentar los cambios realizados y reflexionar sobre su impacto en el análisis.

---

## Paso 1: Cargar el conjunto de datos
En este primer paso vamos a:
- Importar las librerías necesarias (`pandas`, `numpy`).
- Cargar el archivo `productos.xlsx` en un **DataFrame**.
- Visualizar las primeras filas para identificar posibles problemas en las columnas (`precio`, `nombre`, `categoría`).


In [30]:
# =========================================================
# Paso 1: Cargar el conjunto de datos de productos
# =========================================================

# Importamos las librerías necesarias
# import pandas as pd
# import numpy as np

# Cargamos el archivo Excel en un DataFrame
df_prod = pd.read_excel("productos.xlsx")

# Mostramos las primeras filas para revisar la estructura
print("=== Primeras filas del dataset ===")
display(df_prod.head())

# Mostramos información general sobre las columnas y tipos de datos
print("\n=== Información del DataFrame ===")
print(df_prod.info())


=== Primeras filas del dataset ===


,Producto,Categoría,Precio
0,Acero,Metal,$13.54
1,Plexiglás,Polímero/Sintético,$45.10
2,Latón,Metal,$35.64
3,Granito,Piedra Natural,$61.61
4,Acero,Metal,$14.28



=== Información del DataFrame ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Producto   100 non-null    object
 1   Categoría  100 non-null    object
 2   Precio     100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB
None


## Análisis inicial

Después de cargar los datos, vemos que:

- El dataset contiene información básica de productos de un comercio:
  - `Nombre` : texto, con posibles mayúsculas/minúsculas y caracteres especiales.
  - `Precio` : aparece como tipo *object* (texto), lo que indica que no está en formato numérico.
  - `Categoría`: texto, con riesgo de estar desnormalizado (ejemplo: "Electrónica", "electronica", "ELECTRÓNICA").

- El hecho de que **Precio no sea numérico** es un problema importante:
  - No podremos calcular medias, máximos, mínimos ni comparaciones hasta convertirlo a `float`.

- En las columnas de texto (`Nombre`, `Categoría`) puede haber:
  - Variaciones de mayúsculas/minúsculas.
  - Caracteres no deseados (espacios, tildes, símbolos).
  - Inconsistencias en la escritura de categorías.

---

### Conclusión inicial
Antes de analizar precios o agrupar productos por categoría, necesitamos:
1. **Corregir los tipos de datos** de las columnas numéricas.  
2. **Normalizar los campos de texto** para que un mismo producto o categoría no aparezca duplicado en distintas variantes.


In [31]:
# =========================================================
# Paso 2: Corrección del tipo de datos en la columna 'Precio'
# =========================================================

# Volvemos a mirar el tipo de dato actual
print("Tipo de dato actual de 'Precio':", df_prod["Precio"].dtype)

# Intentamos conversión a numérico (valores no convertibles: a NaN)
df_prod["Precio_num"] = pd.to_numeric(df_prod["Precio"], errors="coerce")

# Verificamos si hubo problemas de conversión
problemas = df_prod[df_prod["Precio_num"].isna() & df_prod["Precio"].notna()]

print("\n=== Registros con problemas en la conversión ===")
if not problemas.empty:
    display(problemas)
else:
    print("Todos los valores de 'Precio' se convirtieron correctamente.")

# Mostramos los primeros valores ya convertidos
print("\n=== Vista previa de 'Precio_num' (corregido) ===")
display(df_prod[["Precio", "Precio_num"]].head())


Tipo de dato actual de 'Precio': object

=== Registros con problemas en la conversión ===


,Producto,Categoría,Precio,Precio_num
0,Acero,Metal,$13.54,NaN
1,Plexiglás,Polímero/Sintético,$45.10,NaN
2,Latón,Metal,$35.64,NaN
3,Granito,Piedra Natural,$61.61,NaN
4,Acero,Metal,$14.28,NaN
...,...,...,...,...
95,VIDRIO,Bidrio,$80.31,NaN
96,Granito,Piedra Natural,$10.67,NaN
97,Plexiglás,Polimero/Sintetico,$59.53,NaN
98,Plástico,Polímero/Sintético,$31.13,NaN



=== Vista previa de 'Precio_num' (corregido) ===


,Precio,Precio_num
0,$13.54,NaN
1,$45.10,NaN
2,$35.64,NaN
3,$61.61,NaN
4,$14.28,NaN


### Desastre!

Lo que pasó es que todos los valores de la columna `Precio` tienen el símbolo `$` adelante, y por eso `pd.to_numeric()` no pudo convertirlos y los dejó como `NaN`.

Una posible solución, como mencionamos en la clase del lunes, es limpiar primero los caracteres no deseados (en este caso, el `$`) y recién después convertir a numérico. Vamos!

In [32]:
# =========================================================
# Paso 2 (ver. 2): limpiar y convertir 'Precio' a numérico
# =========================================================

# 1. Quitamos el símbolo "$" y espacios extra
df_prod["Precio_limpio"] = df_prod["Precio"].str.replace("$", "", regex=False).str.strip()

# 2. Convertimos a float
df_prod["Precio_num"] = pd.to_numeric(df_prod["Precio_limpio"], errors="coerce")

# 3. Verificamos el resultado
print("=== Conversión corregida ===")
print(df_prod[["Precio", "Precio_limpio", "Precio_num"]].head(10))

# 4. Chequeamos si todavía quedan problemas
problemas = df_prod[df_prod["Precio_num"].isna() & df_prod["Precio"].notna()]
if not problemas.empty:
    print("\nTodavía hay problemas en algunos registros:")
    display(problemas.head())
else:
    print("\nTodos los precios se convirtieron correctamente.")


=== Conversión corregida ===
   Precio Precio_limpio  Precio_num
0  $13.54         13.54       13.54
1  $45.10         45.10       45.10
2  $35.64         35.64       35.64
3  $61.61         61.61       61.61
4  $14.28         14.28       14.28
5  $77.00         77.00       77.00
6  $93.26         93.26       93.26
7  $86.45         86.45       86.45
8  $89.74         89.74       89.74
9  $42.01         42.01       42.01

Todos los precios se convirtieron correctamente.


#### Mucho mejor!!!!

**Moraleja**: Pandas tiene un montón de métodos "casi mágicos", pero a veces no bastan. Y ahi entra el programador con recursos para buscarle la vuelta al problema. Sigamos!

In [33]:
# =========================================================
# Paso 3: Normalización de texto en las columnas 'Producto'
# y 'Categoría'
# =========================================================

# 1. Normalización de 'Producto'
df_prod["Producto_norm"] = (
    df_prod["Producto"]
    .str.lower()
    .str.strip()
    .str.replace(r"[^a-záéíóúñ\s]", "", regex=True)  # quitamos caracteres no alfabéticos
)

# 2. Normalización de 'Categoría'
# Definimos un diccionario de equivalencias
map_categorias = {
    "metál": "Metal",
    "bidrio": "Vidrio",
    "madera": "Madera",
    "polímero/sintético": "Polímero Sintético",
    "polimero/sintetico": "Polímero Sintético",
    "piedra/natural": "Piedra Natural"
}

df_prod["Categoria_norm"] = (
    df_prod["Categoría"]
    .str.lower()
    .str.strip()
    .replace(map_categorias)
    .str.title()
)

# 3. Vista previa del resultado
print("=== Vista previa de normalización ===")
display(df_prod[["Producto", "Producto_norm", "Categoría", "Categoria_norm"]].head(10))



=== Vista previa de normalización ===


,Producto,Producto_norm,Categoría,Categoria_norm
0,Acero,acero,Metal,Metal
1,Plexiglás,plexiglás,Polímero/Sintético,Polímero Sintético
2,Latón,latón,Metal,Metal
3,Granito,granito,Piedra Natural,Piedra Natural
4,Acero,acero,Metal,Metal
5,Plástico,plástico,Polímero/Sintético,Polímero Sintético
6,MADERA,madera,Madera,Madera
7,Plexiglás,plexiglás,Polímero/Sintético,Polímero Sintético
8,MADERA,madera,Madera,Madera
9,Latón,latón,Metal,Metal


In [34]:
# =========================================================
# Paso 4: Reporte de normalización
# =========================================================

print("=== Comparación de valores únicos antes y después ===\n")

# 1. Categoría
print("Categorías únicas (antes de normalizar):")
print(df_prod["Categoría"].unique())
print(f"Total: {df_prod['Categoría'].nunique()} categorías\n")

print("Categorías únicas (después de normalizar):")
print(df_prod["Categoria_norm"].unique())
print(f"Total: {df_prod['Categoria_norm'].nunique()} categorías\n")

# 2. Producto (solo mostramos conteo porque la lista puede ser larga)
print("Productos únicos (antes de normalizar):", df_prod["Producto"].nunique())
print("Productos únicos (después de normalizar):", df_prod["Producto_norm"].nunique())


=== Comparación de valores únicos antes y después ===

Categorías únicas (antes de normalizar):
['Metal' 'Polímero/Sintético' 'Piedra Natural' 'Madera' 'Vidrio'
 'Piedra/Natural' 'Bidrio' 'Polimero/Sintetico']
Total: 8 categorías

Categorías únicas (después de normalizar):
['Metal' 'Polímero Sintético' 'Piedra Natural' 'Madera' 'Vidrio']
Total: 5 categorías

Productos únicos (antes de normalizar): 11
Productos únicos (después de normalizar): 11


# Fin!!!!

En esta segunda actividad aplicamos métodos de **corrección de tipos de datos** y **normalización** del texto en el dataset de productos.  

Con esto logramos preparar los datos para que resulten consistentes, comparables y listos para análisis más avanzados.


- La **corrección de tipos** permite que la columna `Precio` sea utilizable para cálculos, estadísticas y visualizaciones.  
- La **normalización de texto** elimina inconsistencias y reduce el "ruido" en el dataset, facilitando la segmentación y agrupamiento.  
- Ambos procesos muestran cómo un dataset crudo puede transformarse en una base más limpia, útil y confiable.  

---

> Con este trabajo nos queda un dataset preparado para las siguientes etapas del curso: **transformaciones, filtros y análisis exploratorio**.
